# Cross sectional outliers

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 0: Create properties

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this query, we make properties available to Luminesce
-- by writing them to the Lusid.Property.Definition provider.
-- See the following page for further details:
-- https://support.lusid.com/knowledgebase/article/KA-01702/en-us
-- ===============================================================

-- 1. Define new properties

@newProperties =
values
   ('Instrument/ibor/Sector', 'Text','ibor', 'Sector', 'The sector that the instrument belongs to.'),
   ('Instrument/Fundamentals/pe_ratio','number','Fundamentals','pe_ratio','The P/E ratio.');

@property_definition =
select Column4 as [DisplayName], 'Instrument' as [Domain], Column3 as [PropertyScope], Column4 as [PropertyCode], 'Property' as
   [ConstraintStyle], 'system' as [DataTypeScope], 
   case
      when Column2 == 'Text'
         then 'string'
      else 'number'
      end as [DataTypeCode],
    case
      when Column4 == 'pe_ratio'
         then 'TimeVariant'
      else 'Perpetual'
      end as [Lifetime]
from @newProperties;

-- 2. Write new properties to Lusid.Property.Definition provider

@create_properties =
select *
from Lusid.Property.Definition.Writer
where ToWrite = @property_definition;

-- 3. The results of writing the new property definitions can be seen from the query below:

select *
from @create_properties;


#### Step 1: Upload instrumets

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query we setup some instruments with a P/E property 
-- and a Sector proprerty
-- NOTE: You'll need to have the properties setup as an instrument
-- properties in LUSID and Luminesce as follows:
-- Instrument/ibor/Sector
-- Instrument/Fundamentals/pe_ratio
-- ============================================================

@@Scope = select 'PriceEarningRatioScope';

@instruments_data =

use Drive.Excel
--file=/luminesce-examples/PEdata.xlsx
--worksheet=instrument
enduse;

-- 1. Upload values for custom instrument properties
@ids =
select inst_id as Id
from @instruments_data;

@inst_properties =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, a.PropertyScope as PropertyScope, a.PropertyCode, a.
  Value, @@Scope as EntityScope
from Lusid.Instrument li
inner join (
  select 'pe_ratio' as PropertyCode, 'Fundamentals' as PropertyScope, pe_ratio as Value, inst_id as EntityId
  from @instruments_data
  ) a
  on li.ClientInternal = a.EntityId
where li.ClientInternal in @ids
union
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, a.PropertyScope as PropertyScope, a.PropertyCode, a.
  Value, @@Scope as EntityScope
from Lusid.Instrument li
inner join (
  select 'Sector' as PropertyCode, 'ibor' as PropertyScope, Sector as Value, inst_id as EntityId
  from @instruments_data
  ) a
  on li.ClientInternal = a.EntityId
where li.ClientInternal in @ids;


-- select * from @inst_properties

select *
from Lusid.Property.Writer
where ToWrite = @inst_properties;

-- 2. Upload instrument data to inbuilt properties
-- Transform equity data
@equity_instruments =
select inst_id as ClientInternal, name as DisplayName, ccy as DomCcy, @@Scope as Scope 
from @instruments_data;

-- Write data to Lusid.Instrument.Equity. Print results of writing data to console.
select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;
   
   



#### Step 2: Create cross sectional outlier view

In [ ]:
%%luminesce

-- ==========================================================================
-- Description:
-- Here we build a view which will return all the P/E ratio outliers within
-- a threshold of 2 standard deviations from the mean for a given
-- EffectiveAt/AsAt date and belonging to the group (Sector = Utility)
-- ==========================================================================
-- 1. Create view and set parameters

@cross_sectional_view =
use Sys.Admin.SetupView
--provider=DataQc.CrossSectionalOutlierCheck
--parameters
AsAt,Date,2023-07-06,true
EffectiveAt,Date,2023-07-06,true
StandardDeviations,Int,2,true
PropertyScopes,Text,ibor,true
PropertyKeys,Text,Sector,true
PropertyValues,Text,Utility,true
----
@@StandardDeviations = select  #PARAMETERVALUE(StandardDeviations);
@@Property_Scope = select #PARAMETERVALUE(PropertyScopes);
@@Property_Key = select  #PARAMETERVALUE(PropertyKeys);
@@Property_Value = select  #PARAMETERVALUE(PropertyValues);
@@AsAt = select #PARAMETERVALUE(AsAt);
@@EffectiveAt = select #PARAMETERVALUE(EffectiveAt);

-- 2. Collect instruments within property Scope/Code and get instrument pe_ratio propertys

@custom_props = 
select p.InstrumentId, b.propertycode as PCode, b.Value as PVal
from Lusid.Instrument.Property p
inner join (
select InstrumentId, propertycode, Value
   from Lusid.Instrument.Property p
   where propertyscope = 'Fundamentals' 
   and propertycode = 'pe_ratio') b
   on p.InstrumentId = b.InstrumentId
   where p.propertyscope = @@Property_Scope 
   and p.propertycode = @@Property_Key 
   and p.Value = @@Property_Value ;

@pivoted = 
use Tools.Pivot with @custom_props
--key=PCode
--aggregateColumns=PVal
enduse;

-- 3. Join properties onto Instrument data
@instruments = 
select q.*, p.*
from @pivoted p
inner join (
select *
from Lusid.Instrument
where Scope = 'PriceEarningRatioScope'
) q
on q.LusidInstrumentId = p.InstrumentId;


-- 3. Calculate group mean & SD
@@mean = select avg(pe_ratio) from @instruments;
@sd = select power(avg(power(pe_ratio - @@mean, 2)), 0.5) as x from @instruments;
@@sd = select x from @sd;

@@mean_log = select print('Mean: {X} ', '', 'Logs', @@mean);
@@sd_log = select print('SD: {X} ', '', 'Logs', @@sd);

-- 4. Filter Instruments that are outliers within x Standard Deviations from the mean
select
    InstrumentId,DisplayName,pe_ratio,
    'Outlier' as Result
from @instruments
where abs(pe_ratio - @@mean) >= @@StandardDeviations * @@sd

enduse;

select *
from @cross_sectional_view;



#### Step 3: Run cross sectional outlier view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This is the high level query which an end-user might run
-- to collect all P/E Ratio outliers outside 2 Standard Deviations
-- of the mean belonging to the property group (Sector = Tech) on 
-- an AsAt/EffectiveAt
-- ============================================================

select *
from DataQc.CrossSectionalOutlierCheck
where PropertyScopes = 'ibor'
and PropertyKeys= 'Sector'
and PropertyValues= 'Tech'
and StandardDeviations = 2
and AsAt = '2023-07-06'
and EffectiveAt = '2023-07-06'


